# 📊 Global Superstore Advanced Dashboard

## Overview
This dashboard provides a comprehensive view of the Global Superstore dataset. It allows users to explore historical sales and profit trends, forecast future sales, and analyze key performance metrics interactively.

---

## Key Features

### 1. Filters
- **Region Filter:** Select a specific region or view all regions.
- **Category Filter:** Filter data by product categories such as Furniture, Office Supplies, or Technology.

### 2. Sales & Profit Analysis
- **Sales by Region:** Bar chart showing total sales per region.
- **Profit by Category:** Bar chart displaying total profit per product category.

### 3. Dynamic Forecasting
- **Sales Forecast:** Uses a global NeuralProphet model to predict future sales.
- **Actual + Forecast:** Hover over each point to see actual and forecasted sales values.
- **Forecast Horizon:** Precomputed for 12 months; easily adjustable for longer horizons.
- **Scaled Forecast:** Forecast automatically scales based on selected filter to match historical trends.

### 4. KPI Metrics
- **Total Sales:** Sum of sales for selected filters.
- **Total Profit:** Sum of profits for selected filters.
- **Average Order Value:** Average value of orders based on current selection.

### 5. Interactive & UX Enhancements
- **Light/Dark Mode Switch:** Toggle between light and dark themes for better visual comfort.
- **Export Charts:** Export any chart as PNG or PDF for reporting purposes.
- **Hover Tooltips:** View detailed information for each data point, including actual and forecast values.

---

## How to Use
1. Use the dropdowns to filter by **Region** or **Category**.
2. Explore the **Sales by Region** and **Profit by Category** charts for insights.
3. Navigate to the **Sales Forecast** tab to view future predictions alongside historical sales.
4. Hover over any chart element to see detailed values.
5. Toggle light/dark mode for easier viewing, or export charts for reporting.

---

## Technical Details
- Built with **Dash** and **Plotly** for interactive visualizations.
- Forecasting powered by **NeuralProphet**, leveraging monthly historical sales.
- All logs and warnings suppressed for a clean, fast user experience.
- Global NeuralProphet model reused across filters for lightning-fast performance.

---

## Benefits
- Provides a **holistic view of sales and profit trends**.
- Enables **data-driven decision making** with forecast insights.
- Ideal for **business stakeholders, analysts, and managers** to monitor performance across regions and categories.


In [5]:
# ===============================
# 1. Imports & Logging Setup
# ===============================
import warnings
import logging
warnings.filterwarnings('ignore')
logging.getLogger("neuralprophet").setLevel(logging.ERROR)
logging.getLogger("NP").setLevel(logging.ERROR)

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State
from neuralprophet import NeuralProphet

# ===============================
# 2. Load & Preprocess Data
# ===============================
df = pd.read_csv(r"C:\Users\PMLS\Global_Superstore_Project\Global_Superstore2.csv", encoding="latin1")
df["Order Date"] = pd.to_datetime(df["Order Date"])
df.columns = [c.strip().replace(" ", "_") for c in df.columns]

# Aggregate monthly total sales for global model
monthly_sales = df.groupby(pd.Grouper(key="Order_Date", freq="MS"))[["Sales"]].sum().reset_index()
monthly_sales.columns = ["ds", "y"]

# ===============================
# 3. Fit Single Global NeuralProphet Model
# ===============================
m = NeuralProphet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    epochs=50,
    learning_rate=1.0
)
m.fit(monthly_sales, freq="MS", checkpointing=False)

# Precompute global forecast (max 24 months ahead)
future = m.make_future_dataframe(monthly_sales, periods=24)
global_forecast = m.predict(future)

# ===============================
# 4. Dash App Layout
# ===============================
app = Dash(__name__)

app.layout = html.Div([
    html.H1("📊 Global Superstore Dashboard", style={"textAlign": "center"}),

    # Filters
    html.Div([
        html.Div([
            html.Label("Select Region:"),
            dcc.Dropdown(
                id="region-filter",
                options=[{"label": r, "value": r} for r in df["Region"].unique()],
                value=None,
                placeholder="All Regions"
            )
        ], style={"width": "24%", "display": "inline-block", "marginRight": "1%"}),

        html.Div([
            html.Label("Select Category:"),
            dcc.Dropdown(
                id="category-filter",
                options=[{"label": c, "value": c} for c in df["Category"].unique()],
                value=None,
                placeholder="All Categories"
            )
        ], style={"width": "24%", "display": "inline-block", "marginRight": "1%"}),

        html.Div([
            html.Label("Forecast Horizon (months):"),
            dcc.Dropdown(
                id="forecast-horizon",
                options=[{"label": str(h), "value": h} for h in [6, 12, 24]],
                value=12
            )
        ], style={"width": "24%", "display": "inline-block", "marginRight": "1%"}),

        html.Div([
            html.Label("Mode:"),
            dcc.RadioItems(
                id="mode-switch",
                options=[{"label": "Light", "value": "light"}, {"label": "Dark", "value": "dark"}],
                value="light",
                inline=True
            )
        ], style={"width": "24%", "display": "inline-block"})
    ], style={"marginBottom": "20px"}),

    # KPI cards
    html.Div(id="kpi-cards", style={"display": "flex", "justifyContent": "space-around", "marginBottom": "20px"}),

    # Tabs
    dcc.Tabs([
        dcc.Tab(label="Sales by Region", children=[dcc.Graph(id="sales-region")]),
        dcc.Tab(label="Profit by Category", children=[dcc.Graph(id="profit-category")]),
        dcc.Tab(label="Sales Forecast", children=[
            dcc.Graph(id="sales-forecast"),
            html.Button("Export Forecast PNG", id="export-btn")
        ])
    ])
])

# ===============================
# 5. Callback for Interactive Charts + KPIs
# ===============================
@app.callback(
    Output("sales-region", "figure"),
    Output("profit-category", "figure"),
    Output("sales-forecast", "figure"),
    Output("kpi-cards", "children"),
    Input("region-filter", "value"),
    Input("category-filter", "value"),
    Input("forecast-horizon", "value"),
    Input("mode-switch", "value")
)
def update_charts(region, category, horizon, mode):
    # Filter data
    dff = df.copy()
    if region:
        dff = dff[dff["Region"] == region]
    if category:
        dff = dff[dff["Category"] == category]

    # KPI values
    total_sales = dff["Sales"].sum()
    total_profit = dff["Profit"].sum()
    avg_order = dff["Sales"].mean() if not dff.empty else 0

    kpi_children = [
        html.Div([
            html.H4("Total Sales"),
            html.P(f"${total_sales:,.0f}")
        ], style={"border": "1px solid #ccc", "padding": "10px", "borderRadius": "10px", "width": "30%", "textAlign": "center"}),
        html.Div([
            html.H4("Total Profit"),
            html.P(f"${total_profit:,.0f}")
        ], style={"border": "1px solid #ccc", "padding": "10px", "borderRadius": "10px", "width": "30%", "textAlign": "center"}),
        html.Div([
            html.H4("Avg Order Value"),
            html.P(f"${avg_order:,.2f}")
        ], style={"border": "1px solid #ccc", "padding": "10px", "borderRadius": "10px", "width": "30%", "textAlign": "center"})
    ]

    # Sales by Region
    sales_region = dff.groupby("Region")["Sales"].sum().reset_index()
    fig1 = px.bar(sales_region, x="Region", y="Sales", title="Total Sales by Region")
    if mode == "dark":
        fig1.update_layout(template="plotly_dark")

    # Profit by Category
    profit_category = dff.groupby("Category")["Profit"].sum().reset_index()
    fig2 = px.bar(profit_category, x="Category", y="Profit", title="Total Profit by Category")
    if mode == "dark":
        fig2.update_layout(template="plotly_dark")

    # Sales Forecast
    fig3 = go.Figure()
    actual = dff.groupby(pd.Grouper(key="Order_Date", freq="MS"))[["Sales"]].sum().reset_index()
    actual.columns = ["ds", "y"]

    # Historical line
    if not actual.empty:
        fig3.add_trace(go.Scatter(x=actual["ds"], y=actual["y"], mode="lines+markers",
                                  name="Actual", hovertemplate="%{x|%b %Y}: $%{y:.2f}"))

        # Scale forecast
        scale_factor = actual["y"].iloc[-1] / monthly_sales["y"].iloc[-1]
        scaled_forecast_y = global_forecast["yhat1"][:horizon] * scale_factor
        fig3.add_trace(go.Scatter(x=global_forecast["ds"][:horizon], y=scaled_forecast_y,
                                  mode="lines+markers", name="Forecast (Scaled)",
                                  hovertemplate="%{x|%b %Y}: $%{y:.2f}"))
    else:
        fig3.add_trace(go.Scatter(x=global_forecast["ds"][:horizon], y=global_forecast["yhat1"][:horizon],
                                  mode="lines+markers", name="Forecast",
                                  hovertemplate="%{x|%b %Y}: $%{y:.2f}"))

    fig3.update_layout(title="Sales Forecast", xaxis_title="Date", yaxis_title="Sales")
    if mode == "dark":
        fig3.update_layout(template="plotly_dark")

    return fig1, fig2, fig3, kpi_children

# ===============================
# 6. Run App
# ===============================
if __name__ == "__main__":
    app.run(debug=False, port=8050)


Training: 0it [00:00, ?it/s]

Predicting: 6it [00:00, ?it/s]